## Real Image to Text Annotation Pipeline

This notebook demonstrates using BLIP-2, a state-of-the-art vision-language model by Salesforce, to generate descriptive text annotations (captions) of real images locally in JSON format (defaulting to the test_data/ subdirectory). These annotations can be used as downstream prompts for input into text-to-image diffusion models, to generate new synthetic image datasets. An intermediary large language model (LLM) may be used to format and clean/improve the quality of annotations created before they are used for diffusion input.

HuggingFace docs: https://huggingface.co/docs/transformers/main/en/model_doc/blip_2.

## Set-up environment

First, follow the set up instructions in (make sure you have finished running 'python download_data.py')

Compute advisory: Recommended to run in a GPU environment with high RAM.

In [1]:
%%time

# Standard libraries
import os
import random
import time
import logging
from typing import List, Dict, Tuple, Any
import json

# Third-party libraries
import numpy as np
import PIL
import torch
from transformers import AutoProcessor, Blip2ForConditionalGeneration, BlipProcessor, logging as transformers_logging

# Bitmind-specific libraries
from bitmind.image_dataset import ImageDataset
from bitmind.constants import DATASET_META
from bitmind.constants import IMAGE_ANNOTATION_MODEL
from utils import image_utils

/root/miniconda3/envs/bitmind/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-11 14:32:20.086168: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 14:32:20.229959: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 14:32:20.230948: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-11 14:32:20.466799: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow bin

CPU times: user 5.74 s, sys: 2 s, total: 7.75 s
Wall time: 9.57 s


### Load finetuned BLIP-2 and processor

We can instantiate the model and its corresponding processor from the [hub](https://huggingface.co/models?other=blip-2). Here we load a BLIP-2 checkpoint that leverages the pre-trained OPT model by Meta AI, which as 2.7 billion parameters.

In [2]:
%%time
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

device = "cuda" if torch.cuda.is_available() else "cpu"

#Default log settings
transformers_level = logging.getLogger("transformers").getEffectiveLevel()
huggingface_hub_level = logging.getLogger("huggingface_hub").getEffectiveLevel()

#Suppress logs
logging.getLogger("transformers").setLevel(logging.ERROR)
logging.getLogger("huggingface_hub").setLevel(logging.ERROR)
processor = AutoProcessor.from_pretrained(IMAGE_ANNOTATION_MODEL)
# by default `from_pretrained` loads the weights in float32
# we load in float16 instead to save memory
model = Blip2ForConditionalGeneration.from_pretrained(IMAGE_ANNOTATION_MODEL, torch_dtype=torch.float16) 
model.to(device)

#Restore log settings
logging.getLogger("transformers").setLevel(transformers_level)
logging.getLogger("huggingface_hub").setLevel(huggingface_hub_level)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 2/2 [00:17<00:00,  8.70s/it]


CPU times: user 19.4 s, sys: 17.5 s, total: 36.8 s
Wall time: 39.2 s


### Load Real Image Datasets

In [3]:
%%time
print("Loading real datasets")
real_image_datasets = [
    ImageDataset(ds['path'], 'test', ds.get('name', None), ds['create_splits'])
    for ds in DATASET_META['real']
]
real_image_datasets

Loading real datasets
CPU times: user 7.29 s, sys: 11.4 s, total: 18.7 s
Wall time: 44.8 s


In [4]:
for dataset in real_image_datasets:
    print(dataset.huggingface_dataset_path, dataset.dataset)

dalle-mini/open-images Dataset({
    features: ['url', 'key', 'shard_id', 'status', 'error_message', 'width', 'height', 'exif', 'original_width', 'original_height'],
    num_rows: 125436
})
merkol/ffhq-256 Dataset({
    features: ['image'],
    num_rows: 7000
})
saitsharipov/CelebA-HQ Dataset({
    features: ['image'],
    num_rows: 20260
})


## Prompt-Based Text Annotation

Pipeline for generating text annotations describing each image in real image datasets. 

In [5]:
prompts = [
    "A picture of",
    "The setting is",
    "The background is",
    "The image type/style is"
    # "the background is",
    # "The color(s) are",
    # "The texture(s) are",
    # "The emotion/mood is",
    # "The image medium is",
    # "The image style is"
]

In [6]:
def generate_description(image: PIL.Image.Image, use_prompts: bool = True, verbose: bool = False) -> str:
    """
    Generates a description for a given image using a sequence of prompts.

    Parameters:
    image (PIL.Image.Image): The image for which to generate a text description (string).
    use_prompts (bool) : Determines whether to use prompt input for BLIP-2 image-to-text generation.
    verbose (bool): If True, prints the prompts and answers during processing. Defaults to False.

    Returns:
    str: The generated description for the image.
    """
    if not verbose: transformers_logging.set_verbosity_error() # Only display error messages (no warnings)
    description = ""
    if not use_prompts:
        inputs = processor(image, return_tensors="pt").to(device, torch.float16)
        generated_ids = model.generate(**inputs, max_new_tokens=20)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
        description += generated_text
    else:
        for i, prompt in enumerate(prompts):
            # Append prompt to description to build context history
            description += prompt + ' '
            inputs = processor(image, text=description, return_tensors="pt").to(device, torch.float16)
            generated_ids = model.generate(**inputs, max_new_tokens=20)
            answer = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
            if answer:
                # Append answer to description to build context history
                description += answer
            else:
                description = description[:-len(prompt) - 1]  # Remove the last prompt if no answer is generated
            if verbose:
                print(f"{i}. Prompt: {prompt}")
                print(f"{i}. Answer: {answer}")
    if not verbose: transformers_logging.set_verbosity_info() # Restore transformer warnings
    return description

In [7]:
# Helper functions

def set_logging_level(verbose: int):
    level = logging.WARNING if verbose == 0 else logging.INFO if verbose < 3 else logging.DEBUG
    logging.getLogger().setLevel(level)

def ensure_save_path(path: str) -> str:
    if not os.path.exists(path):
        os.makedirs(path)
    return path

def create_annotation_dataset_directory(base_path: str, dataset_name: str) -> str:
    safe_name = dataset_name.replace("/", "_")
    full_path = os.path.join(base_path, safe_name)
    if not os.path.exists(full_path):
        os.makedirs(full_path)
    return full_path

def generate_annotation(image_id,
                        dataset_name: str,
                        image: PIL.Image.Image,
                        original_dimensions: tuple,
                        resize: bool,
                        resize_dim: int,
                        use_prompts: bool,
                        verbose: int):
    """
    Generate a text annotation for a given image.

    Parameters:
    image_id (int or str): The identifier for the image within the dataset.
    dataset_name (str): The name of the dataset the image belongs to.
    image (PIL.Image.Image): The image object that requires annotation.
    original_dimensions (tuple): Original dimensions of the image as (width, height).
    resize (bool): Allow image downsizing to maximum dimensions of (1280, 1280).
    verbose (int): Verbosity level. If greater than 1, prints image resize message.

    Returns:
    dict: Dictionary containing the annotation data.
    """
    image_to_process = image.copy()
    if resize:
        image_to_process = image_utils.resize_image(image_to_process, resize_dim, resize_dim)
        if verbose > 1 and image_to_process.size != image.size:
            print(f"Resized {image_id}: {image.size} to {image_to_process.size}")

    description = generate_description(image_to_process, use_prompts, verbose > 2)
    annotation = {
        'description': description,
        'original_dataset': dataset_name,
        'original_dimensions': f"{original_dimensions[0]}x{original_dimensions[1]}",
        'index': image_id
    }
    return annotation

def save_annotation(dataset_dir: str, image_id, annotation: dict, verbose: int):
    """
    Save a text annotation to a JSON file if it doesn't already exist.

    Parameters:
    dataset_dir (str): The directory where the annotation file will be saved.
    image_id (int or str): The identifier for the image within the dataset.
    annotation (dict): Annotation data to be saved.
    verbose (int): Verbosity level. If greater than 0, it prints messages during processing.

    Returns:
    int: Returns 0 if the annotation is successfully saved, -1 if the annotation file already exists.
    """
    file_path = os.path.join(dataset_dir, f"{image_id}.json")
    if os.path.exists(file_path):
        if verbose > 0: print(f"Annotation for {image_id} already exists - Skipping")
        return -1  # Skip this image as it already has an annotation
    
    with open(file_path, 'w') as f:
        json.dump(annotation, f, indent=4)
        if verbose > 0: print(f"Created {file_path}")

    return 0

def process_image(dataset_dir: str,
                  image_info: dict,
                  dataset_name: str,
                  image_index: int,
                  resize: bool,
                  resize_dim : int,
                  use_prompts: bool,
                  verbose: int) -> tuple:
    """
    Process an image from a dataset and generate annotations.

    This function handles the processing of an individual image from the given dataset.
    It generates annotations based on the provided parameters, saves the annotation
    in JSON format, and returns the annotation along with the time taken for processing.

    Parameters:
        dataset_dir (str): The directory where the annotation will be stored.
        image_info (dict): Dictionary containing Pillow image.
        dataset_name (str): The name of the dataset.
        image_index (int): The index of the image within the dataset.
        resize (bool): A flag indicating whether the image should be resized.
        resize_dim (int): The dimension to resize the image to if resizing is enabled.
        use_prompts (bool): A flag indicating whether prompts should be used in annotation generation.
        verbose (int): The verbosity level for logging (0 = no logs/messages, 3 = most messages)

    Returns:
        tuple: A tuple containing the generated annotation and the time elapsed during processing.
            If the image data is missing or annotation generation fails, returns (None, time_elapsed).
    """
    if image_info['image'] is None:
        if verbose > 1:
            logging.debug(f"Skipping image {image_index} in dataset {dataset_name} due to missing image data.")
        return None, 0

    original_dimensions = image_info['image'].size
    start_time = time.time()
    annotation = generate_annotation(image_index,
                                     dataset_name,
                                     image_info['image'],
                                     original_dimensions,
                                     resize,
                                     resize_dim,
                                     use_prompts,
                                     verbose)
    save_annotation(dataset_dir, image_index, annotation, verbose)
    time_elapsed = time.time() - start_time

    if annotation == -1:
        if verbose > 1:
            logging.debug(f"Failed to generate annotation for image {image_index} in dataset {dataset_name}")
        return None, time_elapsed
    
    return annotation, time_elapsed

def compute_annotation_latency(processed_images: int, dataset_time: float, dataset_name: str) -> float:
    """
    Compute the average annotation latency for a dataset.

    This function calculates the average time taken to annotate each image in a dataset
    based on the total time spent and the number of processed images. If no images were
    processed, it returns 0.0.

    Parameters:
        processed_images (int): The number of images that were successfully processed.
        dataset_time (float): The total time taken to process the dataset, in seconds.
        dataset_name (str): The name of the dataset.

    Returns:
        float: The average annotation latency per image in seconds. If no images were processed, returns 0.0.
    """
    if processed_images > 0:
        average_latency = dataset_time / processed_images
        logging.info(f'Average annotation latency for {dataset_name}: {average_latency:.4f} seconds')
        return average_latency
    return 0.0

In [8]:
def generate_annotation_dataset(real_image_datasets: List[Any],
                                save_path: str = 'annotations/',
                                verbose: int = 0,
                                max_images: int | None = None,
                                resize_images = False,
                                resize_dim = 512,
                                use_prompts : bool = True) -> Tuple[Dict[str, Dict[str, Any]], float]:
    """
    Generates text annotations for images in the given datasets, saves them in a specified directory, 
    and computes the average per image latency. Returns a dictionary of new annotations and the average latency.

    Parameters:
        real_image_datasets (List[Any]): Datasets containing images.
        save_path (str): Directory path for saving annotation files.
        verbose (int): Verbosity level between 0, 1, 2, 3 for process logs/messages (No messages = 0; Most messages = 3).
        max_images (int): Maximum number of images to annotate.
        resize_images (bool): Allow image downsizing before captioning.
                            Sets max dimensions to (resize_dim, resize_dim), maintaining aspect ratio.
        use_prompts (bool): Whether to enable prompt input for BLIP-2 for more descriptive annotations.
3
    Returns:
        Tuple[Dict[str, Dict[str, Any]], float]: A tuple containing the annotations dictionary and average latency.
    """
    set_logging_level(verbose)
    annotations_dir = ensure_save_path(save_path)
    annotations = {}
    total_time = 0
    total_processed_images = 0

    for i, dataset in enumerate(real_image_datasets):
        dataset_name = dataset.huggingface_dataset_path
        dataset_dir = create_annotation_dataset_directory(annotations_dir, dataset_name)
        processed_images = 0
        dataset_time = 0

        for j, image_info in enumerate(dataset):
            annotation, time_elapsed = \
            process_image(dataset_dir, image_info, dataset_name, j, resize_images, resize_dim, use_prompts, verbose)
            if annotation is not None:
                annotations.setdefault(dataset_name, {})[image_info['id']] = annotation
                total_time += time_elapsed
                dataset_time += time_elapsed
                processed_images += 1
                if max_images is not None and processed_images >= max_images:
                    break

        average_latency = compute_annotation_latency(processed_images, dataset_time, dataset_name)
        total_processed_images += processed_images

    overall_average_latency = total_time / total_processed_images if total_processed_images else 0
    return annotations, overall_average_latency

In [9]:
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
annotations_dict, average_latency = generate_annotation_dataset(real_image_datasets,
                                                                save_path='test_data/annotation_examples/',
                                                                verbose=2,
                                                                max_images=1,
                                                               use_prompts=False)

2024-07-11 14:34:20,967 - INFO - Average annotation latency for dalle-mini/open-images: 9.2043 seconds


Created test_data/annotation_examples/dalle-mini_open-images/1.json


2024-07-11 14:34:25,271 - INFO - Average annotation latency for merkol/ffhq-256: 4.2951 seconds


Created test_data/annotation_examples/merkol_ffhq-256/0.json


2024-07-11 14:34:29,870 - INFO - Average annotation latency for saitsharipov/CelebA-HQ: 4.5956 seconds


Created test_data/annotation_examples/saitsharipov_CelebA-HQ/0.json


In [10]:
annotations_dict

{'dalle-mini/open-images': {'https://farm8.staticflickr.com/7214/7337479734_53f1048393_o.jpg': {'description': 'a man dressed as a witch standing in a field of flowers',
   'original_dataset': 'dalle-mini/open-images',
   'original_dimensions': '612x612',
   'index': 1}},
 'merkol/ffhq-256': {0: {'description': 'a woman talking on a cell phone while wearing a scarf',
   'original_dataset': 'merkol/ffhq-256',
   'original_dimensions': '256x256',
   'index': 0}},
 'saitsharipov/CelebA-HQ': {0: {'description': 'a woman with a smile on her face posing for a picture',
   'original_dataset': 'saitsharipov/CelebA-HQ',
   'original_dimensions': '178x218',
   'index': 0}}}